In [ ]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os
import pickle
import time
import tensorflow as tf
import shutil
import numpy as np
from math import ceil
from src import constants as const
import utils.timer
import helpers

dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
columns_num, = np.where(data_train.dtypes != "object")

timer_path = os.path.join(const.dir.timers(), "timer_tgan.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = utils.timer.Timer()
    timer.save(timer_path)
    del timer
    
from tgan.model import TGANModel

dir_data_gen_tgan = os.path.join(const.dir.data_gen(), "TGAN")
model_dir = os.path.join(const.dir.models(), "tgan")

batch_size = 500
retrain=True

In [ ]:
print(tf.__version__)
tf.test.is_gpu_available()

In [4]:
#?TGANModel

In [5]:
from tqdm.auto import tqdm
from IPython.display import clear_output
from IPython.utils import io

n_synthetic_datasets = 10
overwrite_datasets=False
remove_info_and_warnings=True

if remove_info_and_warnings:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

dataset_dir_tgan = os.path.join(const.dir.data_gen(), "TGAN") 
model_temp_dir = os.path.join(const.dir.models(), "temp_model")
if os.path.exists(model_temp_dir):
    shutil.rmtree(model_temp_dir)
os.makedirs(dataset_dir_tgan, exist_ok = True)
for i in tqdm(range(n_synthetic_datasets), desc = "Generated datasets", disable = False):
    print(i)
    curr_model_path = os.path.join(dataset_dir_tgan, f"gen{i}.csv")
    if overwrite_datasets or not os.path.exists(curr_model_path):
        with io.capture_output() as captured:
            tf.reset_default_graph()
            model_temp_dir_i = os.path.join(model_temp_dir, f"TGAN{i}")
            tgan = TGANModel(columns_num, output = model_temp_dir_i,
                             gpu="0", max_epoch = 100, batch_size=500,
                             steps_per_epoch = data_train.shape[0] / batch_size,
                             num_dis_hidden = 256, num_dis_layers=2, #z_dim=128,
                            restore_session = False, save_checkpoints = True)
            tgan.fit(data_train.reset_index(drop=True))
            fake_train = tgan.sample(ceil(data_train.shape[0] / batch_size) * batch_size).head(data_train.shape[0])
            fake_train.to_csv(curr_model_path)
            del tgan
        clear_output(wait=True)

9
[0317 13:19:46 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[0317 13:19:54 @logger.py:90] Argv: C:\Users\Arne\anaconda3\envs\tfv1_gpu\lib\site-packages\ipykernel_launcher.py -f C:\Users\Arne\AppData\Roaming\jupyter\runtime\kernel-621042e5-cafe-43ef-b7bc-e5c120d46132.json
[0317 13:19:54 @model_utils.py:67] List of Trainable Variables: 
name                              shape         #elements
--------------------------------  ----------  -----------
gen/LSTM/go:0                     [1, 100]            100
gen/LSTM/lstm_cell/kernel:0       [500, 400]       200000
gen/LSTM/lstm_cell/bias:0         [400]               400
gen/LSTM/00/FC/W:0                [100, 100]        10000
gen/LSTM/00/FC/b:0                [100]               100
gen/LSTM/00/FC2/W:0               [100, 1]            100
gen/LSTM/00/FC2/b:0               [1]                   1
gen/LSTM/00/attw:0                [1, 1, 1]             1
gen/LSTM/01/FC/W:0                

[0317 13:19:54 @base.py:209] Setup callbacks graph ...
[0317 13:19:55 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0317 13:19:55 @summary.py:93] Summarizing collection 'summaries' of size 9.
[0317 13:19:55 @graph.py:98] Applying collection UPDATE_OPS of 8 ops.
[0317 13:19:56 @base.py:230] Creating the session ...
[0317 13:19:58 @base.py:236] Initializing the session ...
[0317 13:19:58 @base.py:243] Graph Finalized.
[0317 13:19:58 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...
[0317 13:19:59 @base.py:275] Start Epoch 1 ...
[0317 13:20:12 @base.py:285] Epoch 1 (global_step 60) finished, time:13.2 seconds.
[0317 13:20:14 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-60.
[0317 13:20:14 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.71
[0317 13:20:14 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.348
[0317 13:20:14 @monitor.py:467] GAN_

[0317 13:21:41 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0114
[0317 13:21:41 @monitor.py:467] GAN_loss/gen/klloss: 0.05134
[0317 13:21:42 @monitor.py:467] GAN_loss/gen/loss: 0.96008
[0317 13:21:42 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:21:42 @base.py:275] Start Epoch 11 ...
[0317 13:21:50 @base.py:285] Epoch 11 (global_step 660) finished, time:8.71 seconds.
[0317 13:21:51 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-660.
[0317 13:21:51 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.966
[0317 13:21:51 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.102
[0317 13:21:51 @monitor.py:467] GAN_loss/discrim/loss: 0.64784
[0317 13:21:51 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0027
[0317 13:21:51 @monitor.py:467] GAN_loss/gen/klloss: 0.054348
[0317 13:21:51 @monitor.py:467] GAN_loss/gen/loss: 0.94835
[0317 13:21:51 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:21:51 @base.py:275] Start Epoch 12 ...
[0317 13:2

[0317 13:23:29 @base.py:285] Epoch 21 (global_step 1260) finished, time:8.61 seconds.
[0317 13:23:30 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-1260.
[0317 13:23:30 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.976
[0317 13:23:30 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.09
[0317 13:23:30 @monitor.py:467] GAN_loss/discrim/loss: 0.65579
[0317 13:23:30 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.022
[0317 13:23:30 @monitor.py:467] GAN_loss/gen/klloss: 0.062517
[0317 13:23:30 @monitor.py:467] GAN_loss/gen/loss: 0.95952
[0317 13:23:30 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:23:30 @base.py:275] Start Epoch 22 ...
[0317 13:23:39 @base.py:285] Epoch 22 (global_step 1320) finished, time:8.64 seconds.
[0317 13:23:40 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-1320.
[0317 13:23:40 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.956
[0317 13:23:40 @monitor.py:4

[0317 13:25:10 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.13
[0317 13:25:10 @monitor.py:467] GAN_loss/discrim/loss: 0.63388
[0317 13:25:11 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0273
[0317 13:25:11 @monitor.py:467] GAN_loss/gen/klloss: 0.061282
[0317 13:25:11 @monitor.py:467] GAN_loss/gen/loss: 0.96601
[0317 13:25:11 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:25:11 @base.py:275] Start Epoch 32 ...
[0317 13:25:19 @base.py:285] Epoch 32 (global_step 1920) finished, time:8.59 seconds.
[0317 13:25:20 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-1920.
[0317 13:25:20 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.95
[0317 13:25:20 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.09
[0317 13:25:20 @monitor.py:467] GAN_loss/discrim/loss: 0.64891
[0317 13:25:20 @monitor.py:467] GAN_loss/gen/final-g-loss: 0.98495
[0317 13:25:20 @monitor.py:467] GAN_loss/gen/klloss: 0.043459
[0317 13:25:20 @monitor.py:467] GAN_loss/

...

[0317 13:28:39 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.942
[0317 13:28:39 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.106
[0317 13:28:39 @monitor.py:467] GAN_loss/discrim/loss: 0.65003
[0317 13:28:39 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0171
[0317 13:28:39 @monitor.py:467] GAN_loss/gen/klloss: 0.069605
[0317 13:28:39 @monitor.py:467] GAN_loss/gen/loss: 0.94753
[0317 13:28:39 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:28:39 @base.py:275] Start Epoch 53 ...
[0317 13:28:47 @base.py:285] Epoch 53 (global_step 3180) finished, time:8.76 seconds.
[0317 13:28:48 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-3180.
[0317 13:28:49 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.934
[0317 13:28:49 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.128
[0317 13:28:49 @monitor.py:467] GAN_loss/discrim/loss: 0.64997
[0317 13:28:49 @monitor.py:467] GAN_loss/gen/final-g-loss: 0.99479
[0317 13:28:49 @monitor.py:467

[0317 13:30:18 @monitor.py:467] GAN_loss/gen/klloss: 0.036477
[0317 13:30:18 @monitor.py:467] GAN_loss/gen/loss: 0.99224
[0317 13:30:18 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:30:18 @base.py:275] Start Epoch 63 ...
[0317 13:30:26 @base.py:285] Epoch 63 (global_step 3780) finished, time:8.5 seconds.
[0317 13:30:27 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-3780.
[0317 13:30:27 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.914
[0317 13:30:27 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.278
[0317 13:30:27 @monitor.py:467] GAN_loss/discrim/loss: 0.62533
[0317 13:30:27 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0886
[0317 13:30:27 @monitor.py:467] GAN_loss/gen/klloss: 0.070858
[0317 13:30:27 @monitor.py:467] GAN_loss/gen/loss: 1.0178
[0317 13:30:27 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:30:27 @base.py:275] Start Epoch 64 ...
[0317 13:30:36 @base.py:285] Epoch 64 (global_step 3840) finished, time:8.6

[0317 13:32:05 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-4380.
[0317 13:32:05 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.862
[0317 13:32:05 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.828
[0317 13:32:05 @monitor.py:467] GAN_loss/discrim/loss: 0.42615
[0317 13:32:05 @monitor.py:467] GAN_loss/gen/final-g-loss: 2.1872
[0317 13:32:05 @monitor.py:467] GAN_loss/gen/klloss: 0.079453
[0317 13:32:05 @monitor.py:467] GAN_loss/gen/loss: 2.1077
[0317 13:32:05 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:32:05 @base.py:275] Start Epoch 74 ...
[0317 13:32:14 @base.py:285] Epoch 74 (global_step 4440) finished, time:8.7 seconds.
[0317 13:32:15 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-4440.
[0317 13:32:15 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.912
[0317 13:32:15 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.892
[0317 13:32:15 @monitor.py:467] GAN_loss/dis

[0317 13:33:44 @monitor.py:467] GAN_loss/gen/final-g-loss: 4.1473
[0317 13:33:44 @monitor.py:467] GAN_loss/gen/klloss: 0.19996
[0317 13:33:44 @monitor.py:467] GAN_loss/gen/loss: 3.9474
[0317 13:33:45 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:33:45 @base.py:275] Start Epoch 84 ...
[0317 13:33:53 @base.py:285] Epoch 84 (global_step 5040) finished, time:8.77 seconds.
[0317 13:33:54 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-5040.
[0317 13:33:54 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.936
[0317 13:33:54 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.934
[0317 13:33:54 @monitor.py:467] GAN_loss/discrim/loss: 0.24723
[0317 13:33:55 @monitor.py:467] GAN_loss/gen/final-g-loss: 4.639
[0317 13:33:55 @monitor.py:467] GAN_loss/gen/klloss: 0.16013
[0317 13:33:55 @monitor.py:467] GAN_loss/gen/loss: 4.4789
[0317 13:33:55 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:33:55 @base.py:275] Start Epoch 85 ...
[0317 13:34:

[0317 13:35:31 @base.py:285] Epoch 94 (global_step 5640) finished, time:8.53 seconds.
[0317 13:35:32 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-5640.
[0317 13:35:32 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.98
[0317 13:35:32 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.942
[0317 13:35:33 @monitor.py:467] GAN_loss/discrim/loss: 0.17549
[0317 13:35:33 @monitor.py:467] GAN_loss/gen/final-g-loss: 5.9125
[0317 13:35:33 @monitor.py:467] GAN_loss/gen/klloss: 0.2313
[0317 13:35:33 @monitor.py:467] GAN_loss/gen/loss: 5.6812
[0317 13:35:33 @monitor.py:467] QueueInput/queue_size: 50
[0317 13:35:33 @base.py:275] Start Epoch 95 ...
[0317 13:35:41 @base.py:285] Epoch 95 (global_step 5700) finished, time:8.7 seconds.
[0317 13:35:42 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-5700.
[0317 13:35:42 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.956
[0317 13:35:42 @monitor.py:467]

In [11]:
a = np.sort(np.array([1,-1,2,3,1]))
a

array([-1,  1,  1,  2,  3])

In [6]:
?TGANModel

In [7]:
dat = pd.read_csv(os.path.join(dataset_dir_tgan, "gen8.csv"))
print(dat.shape)
dat

(30162, 15)


,Unnamed: 0,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,0,41.520962,Private,123298.150238,Some-college,9.998246,Married,Transport-moving,White,Male,-0.002,0.002,46.284687,US,<=50K
1,1,29.333905,Private,93412.540380,Bachelors,9.998203,Never-married,Craft-repair,White,Male,-0.002,0.002,39.999070,US,<=50K
2,2,37.155483,Without-pay,348967.585102,Doctorate,5.364805,Never-married,Exec-managerial,White,Male,-0.002,0.002,10.513563,US,<=50K
3,3,27.836704,Private,190716.368825,Some-college,9.998376,Married,Craft-repair,White,Male,-0.002,0.002,39.998848,US,<=50K
4,4,62.593390,Private,177554.013791,HS-grad,8.998297,Never-married,Adm-clerical,White,Male,-0.002,0.002,39.999014,US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,30157,51.908670,Private,79030.448812,HS-grad,8.998333,Never-married,Exec-managerial,White,Female,-0.002,0.002,49.838894,US,<=50K
30158,30158,40.990514,Private,242963.948786,HS-grad,8.998434,Married,Prof-specialty,White,Male,-0.002,0.002,39.998901,US,<=50K
30159,30159,40.361894,Private,361387.180107,HS-grad,8.998331,Married,Adm-clerical,White,Female,-0.002,0.002,31.258797,US,<=50K
30160,30160,41.289241,Private,193311.083292,Some-college,9.998202,Married,Tech-support,White,Male,-0.002,0.002,46.090875,US,<=50K


### 